# Implementing a new model with Jack 

In this tutorial, we focus on the minimal steps required to implement a new model from scratch using Jack.
Please note that this tutorial has a lot of detail. It is aimed at developers who want to understand the internals of Jack. 

In order to implement a Jack Reader, we define three modules:
- **Input Module**: Responsible for mapping `QASetting`s to numpy arrays assoicated with `TensorPort`s
- **Model Module**: Defines the TensorFlow graph (differentiable model architecture)
- **Output Module**: Converting the network output to human-readable overall system output. 

We will implement a simple Bi-LSTM baseline for extractive question answering, which involves extracting the answer string from the given text.
The architecture is as follows:
- Words of question and support are embedded using random embeddings (not trained)
- Both word and question are encoded using a bi-directional LSTM
- The question is summarized by averaging its token representations
- A feedforward NN scores each of the support tokens to be the _start_ of the answer
- A feedforward NN scores each of the support tokens to be the _end_ of the answer

Usually it is enough to implement a _Model_ Module and reuse existing _Input_ and _Output_ modules. We will use the existing `XQAOutputModule`.


In [1]:
# First change dir to jack parent
import os
os.chdir('..')

In [2]:
import re
from jack.core import *
from jack.core.tensorflow import TFReader, TFModelModule
from jack.io.embeddings import Embeddings
from jack.util.hooks import LossHook
from jack.util.vocab import *
from jack.readers.extractive_qa.shared import XQAPorts, XQAOutputModule
from jack.readers.extractive_qa.util import prepare_data
from jack.readers.extractive_qa.util import tokenize
from jack import tfutil
from jack.tfutil import sequence_encoder
from jack.tfutil.misc import mask_for_lengths
from jack.util.map import numpify
from jack.util.preprocessing import stack_and_pad
import tensorflow as tf
_tokenize_pattern = re.compile('\w+|[^\w\s]')

/usr/lib64/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Ports

All communication between _Input_, _Model_ and _Output_ modules happens via `TensorPort`s (see `jack/core/tensorport.py`), which are arrays of a given specific shape.
Normally, you should try to reuse ports wherever possible, because this enables you to reuse previous modules.

If you need a new port, however, it is also straight-forward to define one.
For this tutorial, we will define most ports here.

In [3]:
class MyPorts:

    embedded_question = TensorPort(np.float32, [None, None, None],
                                   "embedded_question",
                                   "Represents the embedded question",
                                   "[B, max_num_question_tokens, N]")
    # or reuse Ports.Misc.embedded_question

    question_length = TensorPort(np.int32, [None],
                                 "question_length",
                                 "Represents length of questions in batch",
                                 "[B]")
    # or reuse Ports.Input.question_length

    embedded_support = TensorPort(np.float32, [None, None, None],
                                  "embedded_support",
                                  "Represents the embedded support",
                                  "[B, max_num_tokens, N]")
    # or reuse Ports.Misc.embedded_support

    support_length = TensorPort(np.int32, [None],
                                "support_length",
                                "Represents length of support in batch",
                                "[B]")
    # or reuse Ports.Input.support_length

    start_scores = TensorPort(np.float32, [None, None],
                              "start_scores",
                              "Represents start scores for each support sequence",
                              "[B, max_num_tokens]")
    # or reuse Ports.Prediction.start_scores

    end_scores = TensorPort(np.float32, [None, None],
                            "end_scores",
                            "Represents end scores for each support sequence",
                            "[B, max_num_tokens]")
    # or reuse Ports.Prediction.end_scores

    span_prediction = TensorPort(np.int32, [None, 2],
                                 "span_prediction",
                                 "Represents predicted answer as a (start, end) span",
                                 "[B, 2]")
    # or reuse Ports.Prediction.span_prediction

    answer_span = TensorPort(np.int32, [None, 2],
                             "answer_span_target",
                             "Represents target answer as a (start, end) span",
                             "[B, 2]")
    # or reuse Ports.Target.answer_span

    token_offsets = TensorPort(np.int32, [None, None],
                               "token_offsets",
                               "Character index of tokens in support.",
                               "[B, support_length]")
    # or reuse XQAPorts.token_offsets
    
    loss = Ports.loss  # this port must be used

## Input Module

The _Input_ module is responsible for converting `QASetting` instances into numpy
arrays, which are mapped to `TensorPort`s and passed on to the _Model_ module.
Effectively, we are building the tensorflow _feed dictionary_ used during training and inference. 
Note, there are _Input_ modules for
several readers that can easily be reused when your model requires the same
pre-processing and input as another model. Similarly, this is also true for the
_Output_ Module. In case you can reuse those modules it is enough to simply
implement a new _Model_ Module (see below) that adheres to the same Tensorport interface.
See `jack/readers/implementations.py` how different readers re-use the same modules.

To implement a new _Input_ module, you could implement the `InputModule` interface, but in many cases it'll be
easier to inherit from `OnlineInputModule`, which already comes with useful functionality. In our implementation, we need to:
- Define the output `TensorPort`s of our input module
- Implement the preprocessing (e.g. tokenization, mapping to embedding vectors, ...). The result of this step is one *annotation* per instance, e.g. a `dict`.
- Implement batching. Given a list of annotations, you need to define how to build the feed dict.

In [4]:
class MyInputModule(OnlineInputModule):
    
    def setup(self):
        self.vocab = self.shared_resources.vocab
        self.emb_matrix = self.vocab.emb.lookup

    # We will now define the input and output TensorPorts of our model.

    @property
    def output_ports(self):
        return [MyPorts.embedded_question,           # Question embeddings
                MyPorts.question_length,             # Lengths of the questions
                MyPorts.embedded_support,            # Support embeddings
                MyPorts.support_length,              # Lengths of the supports
                MyPorts.token_offsets  # Character offsets of tokens in support, used for in ouput module
               ]

    @property
    def training_ports(self):
        return [MyPorts.answer_span]                 # Answer span, one for each question

    # Now, we implement our preprocessing. This involves tokenization,
    # mapping to token IDs, mapping to to token embeddings,
    # and computing the answer spans.

    def _get_emb(self, idx):
        """Maps a token ID to it's respective embedding vector"""
        if idx < self.emb_matrix.shape[0]:
            return self.vocab.emb.lookup[idx]
        else:
            # <OOV>
            return np.zeros([self.vocab.emb_length])

    def preprocess(self, questions, answers=None, is_eval=False):
        """Maps a list of instances to a list of annotations.

        Since in our case, all instances can be preprocessed independently, we'll
        delegate the preprocessing to a `_preprocess_instance()` method.
        """

        if answers is None:
            answers = [None] * len(questions)

        return [self._preprocess_instance(q, a)
                for q, a in zip(questions, answers)]

    def _preprocess_instance(self, question, answers=None):
        """Maps an instance to an annotation.

        An annotation contains the embeddings and length of question and support,
        token offsets, and optionally answer spans.
        """

        has_answers = answers is not None

        # `prepare_data()` handles most of the computation in our case, but
        # you could implement your own preprocessing here.
        q_tokenized, q_ids, _, q_length, s_tokenized, s_ids, _, s_length, \
        word_in_question, offsets, answer_spans = \
            prepare_data(question, answers, self.vocab,
                         with_answers=has_answers,
                         max_support_length=100)
        # there is only 1 support
        s_tokenized, s_ids, s_length, offsets = s_tokenized[0], s_ids[0], s_length[0], offsets[0]

        # For both question and support, we'll fill an embedding tensor
        emb_support = np.zeros([s_length, self.emb_matrix.shape[1]])
        emb_question = np.zeros([q_length, self.emb_matrix.shape[1]])
        for k in range(len(s_ids)):
            emb_support[k] = self._get_emb(s_ids[k])
        for k in range(len(q_ids)):
            emb_question[k] = self._get_emb(q_ids[k])

        # Now, we build the annotation for the question instance. We'll use a
        # dict that maps from `TensorPort` to numpy array, but this could be
        # any data type, like a custom class, or a named tuple.

        annotation = {
            MyPorts.question_length: q_length,
            MyPorts.embedded_question: emb_question,
            MyPorts.support_length: s_length,
            MyPorts.embedded_support: emb_support,
            MyPorts.token_offsets: offsets
        }

        if has_answers:
            # For the purpose of this tutorial, we'll only use the first answer, such
            # that we will have exactly as many answers as questions.
            annotation[MyPorts.answer_span] = answer_spans[0][0]

        return numpify(annotation, keys=annotation.keys())

    def create_batch(self, annotations, is_eval, with_answers):
        """Now, we need to implement the mapping of a list of annotations to a feed dict.
        
        Because our annotations already are dicts mapping TensorPorts to numpy
        arrays, we only need to do padding here.
        """

        return {key: stack_and_pad([a[key] for a in annotations])
                for key in annotations[0].keys()}

## Model Module

The model module defines the TensorFlow computation graph.
It takes _Input_ module outputs as inputs, and produces outputs (such as the loss, or logits)
that match the inputs of the _Output_ module.

In [5]:
class MyModelModule(TFModelModule):

    @property
    def input_ports(self) -> Sequence[TensorPort]:
        return [MyPorts.embedded_question,
                MyPorts.question_length,
                MyPorts.embedded_support,
                MyPorts.support_length]

    @property
    def output_ports(self) -> Sequence[TensorPort]:
        return [MyPorts.start_scores,
                MyPorts.end_scores,
                MyPorts.span_prediction]

    @property
    def training_input_ports(self) -> Sequence[TensorPort]:
        return [MyPorts.start_scores,
                MyPorts.end_scores,
                MyPorts.answer_span]

    @property
    def training_output_ports(self) -> Sequence[TensorPort]:
        return [MyPorts.loss]

    def create_output(self, shared_resources, input_tensors):
        """
        Implements the "core" model: The TensorFlow subgraph which computes the
        answer span from the embedded question and support.
        Args:
            emb_question: [Q, L_q, N]
            question_length: [Q]
            emb_support: [Q, L_s, N]
            support_length: [Q]

        Returns:
            start_scores [B, L_s, N], end_scores [B, L_s, N], span_prediction [B, 2]
        """
        tensors = TensorPortTensors(input_tensors)
        with tf.variable_scope("fast_qa", initializer=tf.contrib.layers.xavier_initializer()):
            dim = shared_resources.config['repr_dim']
            # set shapes for inputs
            tensors.embedded_question.set_shape([None, None, dim])
            tensors.embedded_support.set_shape([None, None, dim])

            # encode question and support
            rnn = tf.contrib.rnn.LSTMBlockFusedCell
            encoded_question = sequence_encoder.bi_lstm(dim, tensors.embedded_question,
                                                        tensors.question_length, name='bilstm',
                                                        with_projection=True)

            encoded_support = sequence_encoder.bi_lstm(dim, tensors.embedded_support,
                                                       tensors.support_length, name='bilstm',
                                                       reuse=True, with_projection=True)

            start_scores, end_scores, predicted_start_pointer, predicted_end_pointer = \
                self._output_layer(dim, encoded_question, tensors.question_length,
                                   encoded_support, tensors.support_length)

            span = tf.concat([predicted_start_pointer, predicted_end_pointer], 1)

            return TensorPort.to_mapping(self.output_ports, (start_scores, end_scores, span))

    def _output_layer(self,
                      dim,
                      encoded_question,
                      question_length,
                      encoded_support,
                      support_length):
        """Simple span prediction layer of our network"""
        batch_size = tf.shape(question_length)[0]

        # Computing weighted question state
        attention_scores = tf.contrib.layers.fully_connected(encoded_question, 1,
                                                             scope="question_attention")
        q_mask = mask_for_lengths(question_length, batch_size)
        attention_scores = attention_scores + tf.expand_dims(q_mask, 2)
        question_attention_weights = tf.nn.softmax(attention_scores, 1,
                                                   name="question_attention_weights")
        question_state = tf.reduce_sum(question_attention_weights * encoded_question, [1])

        # Prediction
        support_mask = mask_for_lengths(support_length, batch_size)
        interaction = tf.expand_dims(question_state, 1) * encoded_support
        
        def predict():
            scores = tf.layers.dense(tf.concat([interaction, encoded_support], axis=2), 1)
            scores = tf.squeeze(scores, [2])
            scores = scores + support_mask
            _, predicted = tf.nn.top_k(scores, 1)
            return scores, predicted

        start_scores, predicted_start_pointer = predict()
        end_scores, predicted_end_pointer = predict()

        return start_scores, end_scores, predicted_start_pointer, predicted_end_pointer

    def create_training_output(self,
                               shared_resources,
                               input_tensors) -> Sequence[TensorPort]:
        """Compute loss from start & end scores and the gold-standard `answer_span`."""
        tensors = TensorPortTensors(input_tensors)
        start, end = [tf.squeeze(t, 1) for t in tf.split(tensors.answer_span_target, 2, 1)]

        start_score_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=tensors.start_scores,
                                                                          labels=start)
        end_score_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=tensors.end_scores,
                                                                        labels=end)
        loss = start_score_loss + end_score_loss
        return TensorPort.to_mapping(self.training_output_ports, [tf.reduce_mean(loss)])

## Output Module

The output module converts our model predictions to `Answer` instances.
Since our model is a standard extractive QA model, and since we used the standard
`TensorPort`s, we can reuse the existing `XQAOutputModule`, rather than implementing
our own.

In [6]:
class MyOutputModule(OutputModule):
    @property
    def input_ports(self) -> List[TensorPort]:
        return [MyPorts.span_prediction,
                MyPorts.token_offsets,
                MyPorts.start_scores,
                MyPorts.end_scores]
    
    def __call__(self,
                 questions,
                 input_tensors) -> Sequence[Answer]:
        """Produces best answer for each question."""
        answers = []
        tensors = TensorPortTensors(input_tensors)
        for i, question in enumerate(questions):
            offsets = tensors.token_offsets[i]
            start, end = tensors.span_prediction[i]
            score = tensors.start_scores[i, start] + tensors.end_scores[i, end]
            # map token to char span
            char_start = offsets[start]
            char_end = offsets[end + 1] if end < len(offsets) - 1 else len(question.support[0])
            answer = question.support[0][char_start: char_end]
            answer = answer.rstrip()
            char_end = char_start + len(answer)
            
            answers.append(Answer(answer, span=(char_start, char_start), score=score))

        return answers

## Training

As a toy example, we will use a dataset of just one example question:

In [7]:
data_set = [
    (QASetting(
        question="Which is it?",
        support=["While b seems plausible, answer a is correct."],
        id="1"),
     [Answer(text="a", span=(32, 33))])
]

Before putting together our newly defined reader, we first define some shared resources. This includes the vocabulary, and a configuration hyperparameter dictionary `config`.

The `build_vocab()` function builds a random embedding matrix. Normally,
we could load pre-trained embeddings here, such as GloVe.

In [8]:
embedding_dim = 10

def build_vocab(questions):
    """Build a vocabulary of random vectors."""

    embedding_lookup = dict()
    for question in questions:
        for t in tokenize(question.question):
            if t not in embedding_lookup:
                embedding_lookup[t] = len(embedding_lookup)
    embeddings = Embeddings(embedding_lookup, 
                            np.random.random([len(embedding_lookup),
                                              embedding_dim]))

    vocab = Vocab(emb=embeddings, init_from_embeddings=True)
    return vocab

questions = [q for q, _ in data_set]
shared_resources = SharedResources(build_vocab(questions),
                                   config={'repr_dim': 10,
                                           'repr_dim_input': embedding_dim})

Now, we'll instantiate all modules with the `shared_resources` as parameter.

In [9]:
tf.reset_default_graph()

input_module = MyInputModule(shared_resources)
model_module = MyModelModule(shared_resources)
output_module = MyOutputModule()

reader = TFReader(shared_resources, input_module, model_module, output_module)

At this point, the Reader is complete! It is composed of the three modules and shared resources, and is now ready to train.

In [10]:
batch_size = 1

hooks = [LossHook(reader, iter_interval=1)]
optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
reader.train(optimizer, data_set, batch_size, max_epochs=10, hooks=hooks)

print()
print(questions[0].question, questions[0].support[0])
answers = reader(questions)
print("{}, {}, {}".format(answers[0].score, answers[0].span, answers[0].text))

INFO:jack.core.reader:Number of parameters: 1943
INFO:jack.core.reader:Start training...
INFO:jack.util.hooks:Epoch 1	Iter 1	train loss 4.620366096496582
INFO:jack.util.hooks:Epoch 2	Iter 2	train loss 3.915457248687744
INFO:jack.util.hooks:Epoch 3	Iter 3	train loss 1.5664793252944946
INFO:jack.util.hooks:Epoch 4	Iter 4	train loss 0.10223503410816193
INFO:jack.util.hooks:Epoch 5	Iter 5	train loss 0.00027284224051982164
INFO:jack.util.hooks:Epoch 6	Iter 6	train loss 0.0001866677193902433
INFO:jack.util.hooks:Epoch 7	Iter 7	train loss 0.0012704157270491123
INFO:jack.util.hooks:Epoch 8	Iter 8	train loss 0.00020085208234377205
INFO:jack.util.hooks:Epoch 9	Iter 9	train loss 1.180166145786643e-05
INFO:jack.util.hooks:Epoch 10	Iter 10	train loss 4.768370445162873e-07

Which is it? While b seems plausible, answer a is correct.
135.45004272460938, (32, 32), a


NOTE: Should you want to train your model with the training script, you need to register the model in `jack.core.implementations`.

## QA model in PyTorch

Now that we implemented a complete reader from scratch. We might want to implement another model reusing as much as possible. We might even want to change frameworks, e.g., use PyTorch instead of TensorFlow.

All we need to do to accomplish this is to write another ModelModule.

Note, the following code requires you to have installed [PyTorch](http://pytorch.org/).

At first we need to define our PyTorch modules that do the computation (independent of jack, although we offer some convenience functions similar to TF).

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from jack.torch_util import embedding, misc, xqa
from jack.torch_util.highway import Highway
from jack.torch_util.rnn import BiLSTM

class MyPredictionTorchModule(nn.Module):
    def __init__(self, shared_resources):
        super(MyPredictionTorchModule, self).__init__()
        self._shared_resources = shared_resources
        repr_dim_input = shared_resources.config["repr_dim_input"]
        repr_dim = shared_resources.config["repr_dim"]
        
        # nn child modules
        self._bilstm = BiLSTM(repr_dim_input, repr_dim)
        self._linear_question_attention = nn.Linear(2 * repr_dim, 1, bias=False)
        self._linear_start_scores = nn.Linear(2 * repr_dim, 1, bias=False)
        self._linear_end_scores = nn.Linear(2 * repr_dim, 1, bias=False)


    def forward(self, emb_question, question_length, emb_support, support_length):
        # encode
        encoded_question = self._bilstm(emb_question)[0]
        encoded_support = self._bilstm(emb_support)[0]

        # answer
        # computing attention over question
        attention_scores = self._linear_question_attention(encoded_question)
        q_mask = misc.mask_for_lengths(question_length)
        attention_scores = attention_scores.squeeze(2) + q_mask
        question_attention_weights = F.softmax(attention_scores, dim=1)
        question_state = torch.matmul(question_attention_weights.unsqueeze(1),
                                      encoded_question).squeeze(1)
        
        interaction = question_state * encoded_support
        # Prediction
        start_scores = self._linear_start_scores(interaction).squeeze(2)
        end_scores = self._linear_start_scores(interaction).squeeze(2)
        # Mask
        support_mask = misc.mask_for_lengths(support_length)
        start_scores += support_mask
        end_scores += support_mask

        _, predicted_start_pointer = start_scores.max(1)
        _, predicted_end_pointer = end_scores.max(1)
        
        # end pointer cannot come before start
        predicted_end_pointer = torch.max(predicted_end_pointer, predicted_start_pointer)

        span = torch.stack([predicted_start_pointer, predicted_end_pointer], 1)
        return start_scores, end_scores, span
    
class MyLossTorchModule(nn.Module):
    def forward(self, start_scores, end_scores, answer_span):
        start, end = answer_span[:, 0], answer_span[:, 1]
        
        # start prediction loss
        loss = -torch.index_select(F.log_softmax(start_scores, dim=1), dim=1, index=start.long())
        # end prediction loss
        loss -= torch.index_select(F.log_softmax(end_scores, dim=1), dim=1, index=end.long())
        
        # mean loss over the current batch
        return loss.mean()

After defining our torch nn.Module classes, we can use them to define our new ModelModule. Note, that the signature of the nn.Module torch implementations above must much the defined tensorport signature in the ModelModule.

In [12]:
from jack.core.torch import PyTorchModelModule, PyTorchReader


class MyTorchModelModule(PyTorchModelModule):

    @property
    def input_ports(self) -> Sequence[TensorPort]:
        return [MyPorts.embedded_question,
                MyPorts.question_length,
                MyPorts.embedded_support,
                MyPorts.support_length]

    @property
    def output_ports(self) -> Sequence[TensorPort]:
        return [MyPorts.start_scores,
                MyPorts.end_scores,
                MyPorts.span_prediction]

    @property
    def training_input_ports(self) -> Sequence[TensorPort]:
        return [MyPorts.start_scores,
                MyPorts.end_scores,
                MyPorts.answer_span]

    @property
    def training_output_ports(self) -> Sequence[TensorPort]:
        return [MyPorts.loss]
    
    
    def create_loss_module(self, shared_resources: SharedResources):
        return MyLossTorchModule()

    def create_prediction_module(self, shared_resources: SharedResources):
        return MyPredictionTorchModule(shared_resources)

After defining our new PyTorchModelModule we can create our JackReader similar as before.

In [13]:
input_module = MyInputModule(shared_resources)
model_module = MyTorchModelModule(shared_resources)  # was MyModelModule
output_module = MyOutputModule()

reader = PyTorchReader(shared_resources, input_module, model_module, output_module)  # was TFReader

At this point it is totally transparent whether we are using a TFReader or a PyTorchReader.

In [14]:
batch_size = 1

# torch needs to be setup already at this point, to get the parameters
reader.setup_from_data(data_set, is_training=True)
optimizer = torch.optim.Adam(reader.model_module.prediction_module.parameters(), lr=0.1)
hooks = [LossHook(reader, iter_interval=1)]
reader.train(optimizer, data_set, batch_size, max_epochs=10, hooks=hooks)

print()
print(questions[0].question, questions[0].support[0])
answers = reader(questions)
print("{}, {}, {}".format(answers[0].score, answers[0].span, answers[0].text))

INFO:jack.core.reader:Setting up data and model...
INFO:jack.core.input_module:OnlineInputModule pre-processes data on-the-fly in first epoch and caches results for subsequent epochs! That means, first epoch might be slower.
INFO:jack.core.reader:Start training...
INFO:jack.util.hooks:Epoch 1	Iter 1	train loss 4.611194133758545
INFO:jack.util.hooks:Epoch 2	Iter 2	train loss 4.488551616668701
INFO:jack.util.hooks:Epoch 3	Iter 3	train loss 4.060502052307129
INFO:jack.util.hooks:Epoch 4	Iter 4	train loss 3.3203110694885254
INFO:jack.util.hooks:Epoch 5	Iter 5	train loss 2.3748998641967773
INFO:jack.util.hooks:Epoch 6	Iter 6	train loss 1.4334105253219604
INFO:jack.util.hooks:Epoch 7	Iter 7	train loss 0.9781996607780457
INFO:jack.util.hooks:Epoch 8	Iter 8	train loss 0.38223710656166077
INFO:jack.util.hooks:Epoch 9	Iter 9	train loss 1.5253791809082031
INFO:jack.util.hooks:Epoch 10	Iter 10	train loss 0.19192257523536682

Which is it? While b seems plausible, answer a is correct.
13.03365898132